# 快速入门 GPT-4 Vison

从历史上看，语言模型系统仅接受**文本**作为输入。但是单一的输入形式，限制了大模型的应用落地范围。

随着技术发展，OpenAI 开发的 GPT-4 Turbo with Vision（简称 GPT-4V）允许模型接收**图像**作为输入，并回答关于它们的问题。

📢注意，目前在 Assistants API 中使用 GPT-4 时还不支持图像输入。

## 使用 GPT-4V 识别线上图像（URL）

![image_sample](https://upload.wikimedia.org/wikipedia/commons/thumb/d/dd/Gfp-wisconsin-madison-the-nature-boardwalk.jpg/2560px-Gfp-wisconsin-madison-the-nature-boardwalk.jpg)

In [10]:
from openai import OpenAI

client = OpenAI(base_url="https://api.xiaoai.plus/v1")

response = client.chat.completions.create(
  model="gpt-4-turbo",
  messages=[
    {
      "role": "user",
      "content": [
        {"type": "text", "text": "介绍下这幅图?"},
        {
          "type": "image_url",
          "image_url": {
            "url": "https://upload.wikimedia.org/wikipedia/commons/thumb/d/dd/Gfp-wisconsin-madison-the-nature-boardwalk.jpg/2560px-Gfp-wisconsin-madison-the-nature-boardwalk.jpg",
          },
        },
      ],
    }
  ],
  max_tokens=300,
)

print(response.choices[0])

Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='这幅图展示了一条木制步道，通向远处的田野和草地。画面充满了自然的绿意，蓝天上点缀着一些白云，显得非常宁静和美丽。两侧的草地绿草如茵，远处的树木和灌木丛也显得生机勃勃。这种场景通常会让人感到放松和愉悦，是一个适合散步或户外活动的好地方。', role='assistant', function_call=None, tool_calls=None))


In [11]:
response.choices[0].message.content

'这幅图展示了一条木制步道，通向远处的田野和草地。画面充满了自然的绿意，蓝天上点缀着一些白云，显得非常宁静和美丽。两侧的草地绿草如茵，远处的树木和灌木丛也显得生机勃勃。这种场景通常会让人感到放松和愉悦，是一个适合散步或户外活动的好地方。'

### 封装成一个函数 query_image_description

In [12]:
def query_image_description(url, prompt="介绍下这幅图?"):
    client = OpenAI(base_url="https://api.xiaoai.plus/v1")  # 初始化 OpenAI 客户端
    
    # 发送请求给 OpenAI 的聊天模型
    response = client.chat.completions.create(
        model="gpt-4-turbo",  # 指定使用的模型
        messages=[
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": prompt},
                    {"type": "image_url", "image_url": {"url": url}},
                ],
            }
        ],
        max_tokens=300,
    )
    
    # 返回模型的响应
    return response.choices[0].message.content


### 调用函数测试

![meme_0](https://p6.itc.cn/q_70/images03/20200602/0c267a0d3d814c9783659eb956969ba1.jpeg)

In [13]:
image_url = "https://p6.itc.cn/q_70/images03/20200602/0c267a0d3d814c9783659eb956969ba1.jpeg"
content = query_image_description(image_url)
print(content)

这幅图是一张幽默的比较图，用于展示狗狗在不同条件下的幻想与现实对比。左边是一张被P图过的狗狗，它被赋予了一副人类肌肉男的身体，十分夸张和不真实，配文为“16岁的我，工作后的我”。右边则是一只正常的柴犬，体态略显臃肿，看起来较为悠闲和惬意，配文描述了与左边的对比，体现了现实中可能的一种自我调侃。

整体来看，这幅图通过夸张和对比的手法，幽默地表达了人们对于理想与现实的反差，以及时间、工作对个体的影响。


### 使用 GPT-4V 识别本地图像文件（Base64编码）


In [14]:
from openai import OpenAI
import base64
import requests
import json

client = OpenAI(base_url="https://api.xiaoai.plus/v1")  # 初始化 OpenAI 客户端

def query_base64_image_description(image_path, prompt="解释下图里的内容？", max_tokens=1000):

    # 实现 Base64 编码
    def encode_image(path):
        with open(path, "rb") as image_file:
            return base64.b64encode(image_file.read()).decode('utf-8')

    # 获取图像的 Base64 编码字符串
    base64_image = encode_image(image_path)

    # 构造请求的 HTTP Header
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {client.api_key}"
    }

    # 构造请求的负载
    payload = {
        "model": "gpt-4-turbo",
        "messages": [
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": prompt},
                    {"type": "image_url", "image_url": {"url": f"data:image/jpeg;base64,{base64_image}"}}
                ]
            }
        ],
        "max_tokens": max_tokens
    }

    # 发送 HTTP 请求
    response = requests.post("https://api.xiaoai.plus/v1/chat/completions", headers=headers, json=payload)

    # 检查响应并提取所需的 content 字段
    if response.status_code == 200:
        response_data = response.json()
        content = response_data['choices'][0]['message']['content']
        return content
    else:
        return f"Error: {response.status_code}, {response.text}"

#### 使用 Assistants API生成的 GDP 40年对比曲线图

![gdp_data](./images/gdp_1980_2020.jpg)

In [15]:
content = query_base64_image_description("./images/gdp_1980_2020.jpg")
print(content)

Error: 400, {"error":{"message":"Invalid content type. image_url is only supported by certain models. (request id: 2024072606040957658217902515252) (request id: 20240726140409525373783UXcKxRN0) (request id: 2024072614040990896126AeHJBRzc)","type":"invalid_request_error","param":"messages.[0].content.[1].type","code":null}}


#### 使用 GPT-4V 识别手写体笔记

![](./images/handwriting_0.jpg)

In [16]:
content = query_base64_image_description("./images/write.png")
print(content)

这张图列举了多个极限求解的公式及其结果。这些公式涵盖了多种函数类型，包括多项式、分式、根号和三角函数，展现了各种不同情况下极限的计算方法。这些内容是高等数学中极限理论的应用，对理解函数的行为在某些点附近的趋势非常重要。

每个公式都使用极限符号“lim”，标明了随着变量（如x或n）趋近某个值时，函数的行为。例如：

1. 图中第一部分的第一行（左侧1），求解当x趋近3时（x→3），函数(x²-9)/(x-3)的极限。通过因式分解得到结果为6。

2. 第二部分利用复合函数的连续性和基础极限求解。(第3号)显示了x→3时，(x²-9)/(x-3)的极限使用x+3代简化后求得6。

3. 第三部分处理了包含无穷大元素和三角函数的复杂情况。(第6号)计算了当x→∞时，(3+5n)sin(x)/(x-cosx)的极限，并通过适当的代换和简化求得结果为1。

这些公式不仅对学习和应用数学极限概念十分有用，也能帮助理解在实际问题中如何运用数学工具来分析和解决问题。


#### 在 Jupyter 标准输出中渲染 Markdown 格式内容

In [17]:
from IPython.display import display, Markdown

# 使用 display 和 Markdown 函数显示 Markdown 内容
display(Markdown(content))

这张图列举了多个极限求解的公式及其结果。这些公式涵盖了多种函数类型，包括多项式、分式、根号和三角函数，展现了各种不同情况下极限的计算方法。这些内容是高等数学中极限理论的应用，对理解函数的行为在某些点附近的趋势非常重要。

每个公式都使用极限符号“lim”，标明了随着变量（如x或n）趋近某个值时，函数的行为。例如：

1. 图中第一部分的第一行（左侧1），求解当x趋近3时（x→3），函数(x²-9)/(x-3)的极限。通过因式分解得到结果为6。

2. 第二部分利用复合函数的连续性和基础极限求解。(第3号)显示了x→3时，(x²-9)/(x-3)的极限使用x+3代简化后求得6。

3. 第三部分处理了包含无穷大元素和三角函数的复杂情况。(第6号)计算了当x→∞时，(3+5n)sin(x)/(x-cosx)的极限，并通过适当的代换和简化求得结果为1。

这些公式不仅对学习和应用数学极限概念十分有用，也能帮助理解在实际问题中如何运用数学工具来分析和解决问题。

![](./images/handwriting_1.jpg)

In [21]:
content = query_base64_image_description("./images/handwriting_1.jpg")
display(Markdown(content))

这幅图片中展示的是一本笔记本的两页，记录了一些关于深度学习、尤其是变换器模型（Transformers），以及不同训练方法和技术的笔记。具体来看：

左侧页面对Transformer模型进行了总结，提到了一些关于参数效率化转换（PEFT）的SOTA（最先进技术）和PBFT的方法。页面下半部分详细列举了不同的prompt tuning技巧，如Adapter（Google, 2019）、Prefix（Stanford, 2021）、P-tuning等。还提到了某些指令式fine-tuning，如GPT-NeoX，GPT-J，GPT-Neo，Chinchilla，还提到了一些具体模型如ELeutherAI的LLMs。

右侧页面继续了类似的主题，聚焦在多模态和指令式fine-tuning技术上，提到了LaMDA (LLaMa)等模型，新的技术如LoRA、PETC以及其他的一些技术创新。详细记录了如Prefix-tuning、Adapters及其在大型语言模型（LLMs）中的应用。

这些笔记可能是某位研究者或学生在学习或研究人工智能、机器学习领域最新技术时的记录。内容涵盖了最前沿的技术与理论，反映了深度学习特别是自然语言处理（NLP）领域的最新发展。

## Homework: 


### #1

使用 GPT-4V 识别带有手写体文字的本地图像文件，分享结果。

### #2

整合 `query_base64_image_description` 函数和 Markdown 格式渲染方法，使得输出结果更易阅读。

In [22]:
gpt_v4_content = query_base64_image_description("./images/gpt_4v.jpg")
print(gpt_v4_content)

这幅图主要是数列与级数的总结，包括基本定义、特殊的数列类型、常见级数及其性质等内容。详细内容如下：

1. **序列与函数关系**：
    - 序列 \(u_n\) 定义为 \(u_n = f(n)\)，其中 \(f\) 为函数，\(n \geq m\)（\(m\) 通常取 1）。
    - 若函数 \(F(a_0, a_1, ..., a_n) = 0\) 且满足 \(n \geq m\)，则称它为函数关系。

2. **序列的特殊类型**：
    - **有界序列**：\(m \leq a_n \leq M\)，即所有序列项都在 \(m\) 和 \(M\) 之间。
    - **增减序列**：单调增加（\(a_{n+1} > a_n\)）或单调减少（\(a_{n+1} < a_n\)）。
    - **上界与下界**：上界为序列中的最小最大值，下界为序列中的最大最小值。

3. **级数**：
    - **算术级数/等差级数 (A.P.)**：每一项与前一项之差为常数 \(d\)，即 \(a_{n+1} = a_n + d\)。
    - **几何级数/等比级数 (G.P.)**：每一项与前一项之比为常数 \(q\)，即 \(a_{n+1} = a_n \times q\)。
    - 级数的求和公式也被列出，例如等差级数和为 \(S_n = \frac{n}{2} [2a+(n-1)d]\) 和等比级数和为 \(S_n = a \frac{1-q^n}{1-q}\)（当 \(q \neq 1\)）。

4. **特殊序列公式**：
    - \(a_n = n^a\)，其中 \(a\) 是常数。
    - 以及 \(P(x)\) 形式的序列，如多项式 \(a_n = n^p - \frac{1}{n^r}\) 的情况，等等。

这幅图为学习高中数学或大学入门级数学序列和级数的概念和公式提供了一个很好的图形化总结，适合用作复习和参考。


In [23]:
from IPython.display import display, Markdown

# 使用 display 和 Markdown 函数显示 Markdown 内容
display(Markdown(gpt_v4_content))

这幅图主要是数列与级数的总结，包括基本定义、特殊的数列类型、常见级数及其性质等内容。详细内容如下：

1. **序列与函数关系**：
    - 序列 \(u_n\) 定义为 \(u_n = f(n)\)，其中 \(f\) 为函数，\(n \geq m\)（\(m\) 通常取 1）。
    - 若函数 \(F(a_0, a_1, ..., a_n) = 0\) 且满足 \(n \geq m\)，则称它为函数关系。

2. **序列的特殊类型**：
    - **有界序列**：\(m \leq a_n \leq M\)，即所有序列项都在 \(m\) 和 \(M\) 之间。
    - **增减序列**：单调增加（\(a_{n+1} > a_n\)）或单调减少（\(a_{n+1} < a_n\)）。
    - **上界与下界**：上界为序列中的最小最大值，下界为序列中的最大最小值。

3. **级数**：
    - **算术级数/等差级数 (A.P.)**：每一项与前一项之差为常数 \(d\)，即 \(a_{n+1} = a_n + d\)。
    - **几何级数/等比级数 (G.P.)**：每一项与前一项之比为常数 \(q\)，即 \(a_{n+1} = a_n \times q\)。
    - 级数的求和公式也被列出，例如等差级数和为 \(S_n = \frac{n}{2} [2a+(n-1)d]\) 和等比级数和为 \(S_n = a \frac{1-q^n}{1-q}\)（当 \(q \neq 1\)）。

4. **特殊序列公式**：
    - \(a_n = n^a\)，其中 \(a\) 是常数。
    - 以及 \(P(x)\) 形式的序列，如多项式 \(a_n = n^p - \frac{1}{n^r}\) 的情况，等等。

这幅图为学习高中数学或大学入门级数学序列和级数的概念和公式提供了一个很好的图形化总结，适合用作复习和参考。